In [3]:
import docker
import pandas as pd

def docker_to_dataframe():
    # Create a Docker client connected to the local Docker daemon
    client = docker.from_env()
    
    # Retrieve a list of images
    images = client.images.list()
    
    # Prepare data for DataFrame
    image_data = []
    for image in images:
        for tag in image.tags:
            repo, tag = tag.split(":") if ":" in tag else (tag, "latest")
            image_data.append({
                "REPOSITORY": repo,
                "TAG": tag,
                "IMAGE ID": image.id.split(":")[1][:12],
                "CREATED": image.attrs['Created'],
                "SIZE": image.attrs['Size']
            })
    
    # Create a DataFrame from the image data
    df1 = pd.DataFrame(image_data)
    
    containers = client.containers.list(all=True)
    
    # Prepare data for DataFrame
    container_data = []
    for container in containers:
        container_data.append({
            "CONTAINER ID": container.short_id,
            "IMAGE": container.image.tags[0] if container.image.tags else 'No tag',
            "COMMAND": container.attrs['Config']['Cmd'],
            "CREATED": container.attrs['Created'],
            "STATUS": container.status,
            "NAMES": container.name
        })
    
    # Create a DataFrame from the container data
    df2 = pd.DataFrame(container_data)
    
    return df1,df2

# Call the function and display the DataFrame
df1,df2 = docker_images_to_dataframe()

In [6]:
df1['IMAGE'] = df1['REPOSITORY']+":"+df1['TAG'] 

In [7]:
df1

,REPOSITORY,TAG,IMAGE ID,CREATED,SIZE,IMAGE
0,faris_test_v1,latest,8b887e727987,2024-09-18T10:25:19.179300017+03:00,823389415,faris_test_v1:latest
1,hello-world,latest,d2c94e258dcb,2023-05-02T16:49:27Z,13256,hello-world:latest


In [11]:
df2.merge(df1,on='IMAGE',how='left',suffixes=('_CONTAINER','_IMAGE'))

,CONTAINER ID,IMAGE,COMMAND,CREATED_CONTAINER,STATUS,NAMES,REPOSITORY,TAG,IMAGE ID,CREATED_IMAGE,SIZE
0,dae48ece0e90,hello-world:latest,[/hello],2024-09-18T11:46:21.250981946Z,exited,optimistic_wu,hello-world,latest,d2c94e258dcb,2023-05-02T16:49:27Z,13256
1,331b166feb1e,faris_test_v1:latest,"[uvicorn, main:app, --host, 0.0.0.0, --port, 8...",2024-09-18T07:25:29.215156412Z,exited,faris_test_v1,faris_test_v1,latest,8b887e727987,2024-09-18T10:25:19.179300017+03:00,823389415
